# 转换并量化中文LLaMA/Alpaca模型

🎉🎉🎉 **新：现在免费用户也有机会能够转换7B和13B模型了！**

💡 提示和小窍门：
- 免费用户默认的内存只有12G左右，**笔者用免费账号实测选择TPU的话有机会随机出35G内存**，建议多试几次。如果能随机出25G内存以上的机器就可以了转换7B模型了，35G内存以上机器就能转换13B模型了
- Pro(+)用户请选择 “代码执行程序” -> “更改运行时类型” -> “高RAM”
- 实测：转换7B级别模型，25G内存的机器就够了；转换13B级别模型需要30G以上的内存（程序莫名崩掉或断开连接就说明内存爆了）
- 如果选了“高RAM”之后内存还是不够大的话，选择以下操作，有的时候会分配出很高内存的机器，祝你好运😄！
    - 可以把GPU或者TPU也选上（虽然不会用到）
    - 选GPU时，Pro用户可选“高级”类型GPU

以下信息配置信息供参考（Pro订阅下测试），运行时规格设置为“高RAM”时的设备配置如下（有随机性）：

| 硬件加速器  |  RAM  |  硬盘  |
| :-- | :--: | :--: |
| None | 25GB | 225GB |
| TPU | 35GB | 225GB |
| GPU（标准，T4）| 25GB | 166GB |
| GPU（高性能，V100）| 25GB | 166GB |
| GPU（高性能，A100）| **80GB** | 166GB |

*温馨提示：用完之后注意断开运行时，选择满足要求的最低配置即可，避免不必要的计算单元消耗（Pro只给100个计算单元）。*

## 安装相关依赖

In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install peft
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-qrvkcgfv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-qrvkcgfv
  Resolved https://github.com/huggingface/transformers.git to commit 4c01231e67f0d699e0236c11178c956fb9753a17
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.28.0.dev0-py3-none-any.whl size=6911490 sha256=c1484b91f5ef8feccdfa356e11e61d5f16be66f44c10ee8370398bb3a969f907
  Stored in directory: /tmp/pip-ephem-wheel-cache-sa1w67v2/wheels/f7/92/8c/752ff3bfcd3439805d8bbf641614da3

## 克隆目录和代码

In [2]:
!git clone https://github.com/ymcui/Chinese-LLaMA-Alpaca
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'Chinese-LLaMA-Alpaca'...
remote: Enumerating objects: 479, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 479 (delta 94), reused 138 (delta 79), pack-reused 269
Receiving objects: 100% (479/479), 10.68 MiB | 25.91 MiB/s, done.
Resolving deltas: 100% (279/279), done.
Cloning into 'llama.cpp'...
remote: Enumerating objects: 1503, done.
remote: Counting objects: 100% (564/564), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 1503 (delta 520), reused 508 (delta 501), pack-reused 939
Receiving objects: 100% (1503/1503), 1.79 MiB | 14.58 MiB/s, done.
Resolving deltas: 100% (945/945), done.


## 合并模型（以Alpaca-7B为例）

**⚠️ 再次提醒：7B模型需要25G内存，13B模型需要35G+内存。**

此处使用的是🤗模型库中提供的基模型（已是HF格式），而不是Facebook官方的LLaMA模型，因此略去将原版LLaMA转换为HF格式的步骤。

**这里直接运行第二步：合并LoRA权重**，生成全量模型权重。可以直接指定🤗模型库的地址，也可以是本地存放地址。
- 基模型：`decapoda-research/llama-7b-hf` *（use at your own risk）*
- LoRA模型：`ziqingyang/chinese-alpaca-lora-7b`

💡 转换13B模型提示：
- 请将参数`--base_model`和`--lora_model`中的的`7b`改为`13b`即可
- **免费用户必须增加一个参数`--offload_dir`以缓解内存压力**，例如`--offload_dir ./offload_temp`

该过程比较耗时（下载+转换），需要几分钟到十几分钟不等，请耐心等待。
转换好的模型存放在`alpaca-combined`目录。
如果你不需要量化模型，那么到这一步就结束了。

In [3]:
!python ./Chinese-LLaMA-Alpaca/scripts/merge_llama_with_chinese_lora.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --lora_model 'ziqingyang/chinese-alpaca-lora-7b' \
    --output_dir alpaca-combined

2023-04-11 10:06:51.752004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100% 33/33 [00:15<00:00,  2.16it/s]
Extended vocabulary size: 49954
Loading LoRA for 7B model
Peft version: 0.2.0
Merging model
Saving shard 1 of 1 into alpaca-combined/consolidated.00.pth


## 量化模型
接下来我们使用[llama.cpp](https://github.com/ggerganov/llama.cpp)工具对上一步生成的全量版本权重进行转换，生成4-bit量化模型。

### 编译工具

首先对llama.cpp工具进行编译。

In [4]:
!cd llama.cpp && make

I llama.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -DNDEBUG -std=c11   -fPIC -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -Wno-unused-function -pthread -march=native -mtune=native
I CXXFLAGS: -I. -I./examples -O3 -DNDEBUG -std=c++11 -fPIC -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native
I LDFLAGS:  
I CC:       cc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
I CXX:      g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0

cc  -I.              -O3 -DNDEBUG -std=c11   -fPIC -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -Wno-unused-function -pthread -march=native -mtune=native   -c ggml.c -o ggml.o
g++ -I. -I./examples -O3 -DNDEBUG -std=c++11 -fPIC -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native -c llama.cpp -o ll

### 模型转换为ggml格式（FP16）

这一步，我们将模型转换为ggml格式（FP16）。
- 在这之前需要把`alpaca-combined`目录挪个位置，并且保证符合转换脚本的要求。
- tokenizer文件需要在模型文件的父节点上（注意这里使用的是中文Alpaca模型附带的文件，而不是合并模型步骤转换出来的）。
- 这里我们直接从 https://huggingface.co/ziqingyang/chinese-alpaca-lora-7b/resolve/main/tokenizer.model 下载中文Alpaca-7B的tokenizer.model文件。

💡 转换13B模型提示：
- tokenizer可以直接用7B的，13B和7B的相同
- llama和alpaca的tokenizer不可混用
- 以下看到7B字样的都是文件夹名，与转换过程没有关系了，改不改都行

In [5]:
!cd llama.cpp && mkdir zh-models && mv ../alpaca-combined zh-models/7B

In [6]:
!cd llama.cpp/zh-models && wget https://huggingface.co/ziqingyang/chinese-alpaca-lora-7b/resolve/main/tokenizer.model

--2023-04-11 10:12:13--  https://huggingface.co/ziqingyang/chinese-alpaca-lora-7b/resolve/main/tokenizer.model
Resolving huggingface.co (huggingface.co)... 13.249.85.92, 13.249.85.69, 13.249.85.16, ...
Connecting to huggingface.co (huggingface.co)|13.249.85.92|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/0f/01/0f01544c04c27e0a0357540e7be5763000a215cedb3be4a0356b56983f2fd5e3/2d967e855b1213a439df6c8ce2791f869c84b4f3b6cfacf22b86440b8192a2f8?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27tokenizer.model%3B+filename%3D%22tokenizer.model%22%3B&Expires=1681466367&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzBmLzAxLzBmMDE1NDRjMDRjMjdlMGEwMzU3NTQwZTdiZTU3NjMwMDBhMjE1Y2VkYjNiZTRhMDM1NmI1Njk4M2YyZmQ1ZTMvMmQ5NjdlODU1YjEyMTNhNDM5ZGY2YzhjZTI3OTFmODY5Yzg0YjRmM2I2Y2ZhY2YyMmI4NjQ0MGI4MTkyYTJmOD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NU

In [7]:
!cd llama.cpp && python convert-pth-to-ggml.py zh-models/7B/ 1

{'dim': 4096, 'multiple_of': 256, 'n_heads': 32, 'n_layers': 32, 'norm_eps': 1e-06, 'vocab_size': -1}
Namespace(dir_model='zh-models/7B/', ftype=1, vocab_only=0)
n_parts = 1

Processing part 1 of 1

Processing variable: tok_embeddings.weight with shape: (49954, 4096) and type: torch.float16
Processing variable: layers.0.attention.wq.weight with shape: (4096, 4096) and type: torch.float16
Processing variable: layers.0.attention.wk.weight with shape: (4096, 4096) and type: torch.float16
Processing variable: layers.0.attention.wv.weight with shape: (4096, 4096) and type: torch.float16
Processing variable: layers.0.attention.wo.weight with shape: (4096, 4096) and type: torch.float16
Processing variable: layers.0.feed_forward.w1.weight with shape: (11008, 4096) and type: torch.float16
Processing variable: layers.0.feed_forward.w2.weight with shape: (4096, 11008) and type: torch.float16
Processing variable: layers.0.feed_forward.w3.weight with shape: (11008, 4096) and type: torch.float16
Pro

### 将FP16模型量化为4-bit

**⚠️ 本步骤消耗内存峰值为<font size="5">3-4G</font>左右，运行前务必确认是否有足够空闲内存！**

我们进一步将FP16模型转换为4-bit量化模型。

In [8]:
!cd llama.cpp && ./quantize ./zh-models/7B/ggml-model-f16.bin ./zh-models/7B/ggml-model-q4_0.bin 2

llama.cpp: loading model from ./zh-models/7B/ggml-model-f16.bin
llama.cpp: saving model to ./zh-models/7B/ggml-model-q4_0.bin
[1/291]                tok_embeddings.weight - [4096 x 49954], type =    f16, quantizing .. size =   390.27 MB ->   121.96 MB | hist: 0.000 0.022 0.019 0.033 0.053 0.078 0.104 0.125 0.133 0.125 0.104 0.078 0.053 0.033 0.019 0.022 
[2/291]         layers.0.attention.wq.weight - [4096 x 4096], type =    f16, quantizing .. size =    32.00 MB ->    10.00 MB | hist: 0.000 0.021 0.016 0.028 0.046 0.071 0.103 0.137 0.158 0.137 0.103 0.071 0.046 0.028 0.016 0.021 
[3/291]         layers.0.attention.wk.weight - [4096 x 4096], type =    f16, quantizing .. size =    32.00 MB ->    10.00 MB | hist: 0.000 0.021 0.016 0.027 0.045 0.071 0.104 0.138 0.158 0.139 0.104 0.071 0.045 0.027 0.016 0.021 
[4/291]         layers.0.attention.wv.weight - [4096 x 4096], type =    f16, quantizing .. size =    32.00 MB ->    10.00 MB | hist: 0.000 0.022 0.018 0.032 0.051 0.076 0.103 0.128 0.

### （可选）测试量化模型解码
至此已完成了所有转换步骤。
我们运行一条命令测试一下是否能够正常加载并进行对话。

FP16和Q4量化文件存放在./llama.cpp/zh-models/7B下，可按需下载使用。

In [9]:
!cd llama.cpp && ./main -m ./zh-models/7B/ggml-model-q4_0.bin --color -f ./prompts/alpaca.txt -p "详细介绍一下北京的名胜古迹：" -n 512

main: seed = 1681208198
llama.cpp: loading model from ./zh-models/7B/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 49954
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: f16        = 2
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  59.11 KB
llama_model_load_internal: mem required  = 5896.99 MB (+ 1026.00 MB per state)
llama_init_from_file: kv self size  =  256.00 MB

system_info: n_threads = 40 / 40 | AVX = 1 | AVX2 = 1 | AVX512 = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
sampling: te